### Word Videos Select

In [39]:
import os
import sys
import numpy as np
import pandas as pd
import glob
from gtts import gTTS
from PIL import Image, ImageFilter, ImageOps
#from playsound import playsound
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, concatenate_audioclips,CompositeVideoClip, CompositeAudioClip
import pandas as pd
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil
from os.path import isfile, join
import time
import cv2

In [40]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [41]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [42]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Read Data

In [43]:
# image video path folder
group_num = 1 # 1,2,3
# word
word_video_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/1000 Words/Deployment3/Word{group_num}/Result/Image_Video_Audio"

In [44]:
df_deployment_word_master_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/\
2-1-Word Prefix Suffix Analysis/{lang_folder.capitalize()}_1000_Word_Person_Case_Tense_Plural_Mean_Manuel.xlsx", sheet_name="Sheet1")
df_deployment_word_master_file.drop_duplicates(inplace=True)
df_deployment_word_master_file.reset_index(drop=True, inplace=True)
df_deployment_word_master_file

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
0,bir,NUM,NaN,bir,bir,bir,a,a,18835735,+,NaN,NaN,NaN,NaN,NaN,NaN
1,bir,ADV,NaN,biraz,biraz,bir,a little,a little,1269641,+az,NaN,NaN,NaN,NaN,NaN,NaN
2,bir,PRON,NaN,biri,biri,bir,somebody,somebody,837400,+i,NaN,NaN,belirtme,NaN,NaN,NaN
3,bir,ADV,NaN,birlikte,birlikte,bir,together,together,409940,+likte,NaN,NaN,NaN,NaN,NaN,NaN
4,bir,NOUN,NaN,birini,biri,bir,one,somebody,259916,+ini,NaN,NaN,belirtme,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,veriyor,VERB,NaN,veriyorum,ver,ver,i give,give,126290,+um,NaN,1.tekil,NaN,NaN,NaN,NaN
1620,veriyor,VERB,NaN,veriyor,ver,ver,giving,give,68163,+,şimdiki,NaN,NaN,NaN,NaN,NaN
1621,okul,NOUN,NaN,okul,okul,oku,school,school,68160,+,NaN,NaN,NaN,NaN,NaN,NaN
1622,suçlu,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124,+,NaN,3.tekil,belirtme,NaN,NaN,NaN


In [45]:
df_deployment_ngram_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/Manuel Select/\
Revised Ngrams All in One.xlsx", sheet_name="Sheet2")
df_deployment_ngram_file.drop_duplicates(inplace=True)
df_deployment_ngram_file.reset_index(drop=True, inplace=True)
df_deployment_ngram_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,yardım eder misin,eder misin
2,bana yardım eder misin,yardım eder misin,yardım eder
3,başka bir şey daha,başka bir şey,bir şey
4,başka bir şey daha,başka bir şey,başka bir
...,...,...,...
475,NaN,çok kötü bir,çok kötü
476,NaN,bir şey olmaz,NaN
477,NaN,şuna bir bak,bir bak
478,NaN,şuna bir bak,NaN


In [46]:
df_word_used_ngram_count = word_count_result(df_deployment_ngram_file, ["fourgram","threegram","twogram"], set_condition=True)
df_word_used_ngram_count

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,word,word_count
0,bir,124
1,şey,90
2,ne,72
3,var,46
4,bu,36
...,...,...
155,peki,1
156,dur,1
157,pek,1
158,kimsin,1


In [47]:
used_word_list = list(df_word_used_ngram_count["word"])
#used_word_list

In [48]:
df_deployment_word_remain = df_deployment_word_master_file[~df_deployment_word_master_file["word"].isin(used_word_list)]
df_deployment_word_remain

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
1,bir,ADV,NaN,biraz,biraz,bir,a little,a little,1269641,+az,NaN,NaN,NaN,NaN,NaN,NaN
2,bir,PRON,NaN,biri,biri,bir,somebody,somebody,837400,+i,NaN,NaN,belirtme,NaN,NaN,NaN
3,bir,ADV,NaN,birlikte,birlikte,bir,together,together,409940,+likte,NaN,NaN,NaN,NaN,NaN,NaN
4,bir,NOUN,NaN,birini,biri,bir,one,somebody,259916,+ini,NaN,NaN,belirtme,NaN,NaN,NaN
5,bir,NOUN,NaN,birisi,biri,bir,someone,somebody,182370,+isi,NaN,NaN,belirtme,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,veriyor,VERB,NaN,veriyorum,ver,ver,i give,give,126290,+um,NaN,1.tekil,NaN,NaN,NaN,NaN
1620,veriyor,VERB,NaN,veriyor,ver,ver,giving,give,68163,+,şimdiki,NaN,NaN,NaN,NaN,NaN
1621,okul,NOUN,NaN,okul,okul,oku,school,school,68160,+,NaN,NaN,NaN,NaN,NaN,NaN
1622,suçlu,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124,+,NaN,3.tekil,belirtme,NaN,NaN,NaN


#### Output Path Folder File

In [49]:
# Output path and folder ???????
result_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words"
main_result_folder = "Deployment3 Select"  # Fourgram_Selected, Test2
result_folder = f"Word Video Select"  # Twogram, Threegram, Fourgram, Sample


In [50]:
# ///***
output_path = f"{result_path}/{main_result_folder}/{result_folder}"  # output path

Path(output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

##### Tenses

###### Past

In [51]:
# ///***
add_past_out_folder = "Geçmiş"

past_output_path = f"{output_path}/{add_past_out_folder}"

Path(past_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [52]:
df_past_tense_result = df_deployment_word_remain[df_deployment_word_remain["tense"] == "geçmiş"]
df_past_tense_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
57,de,VERB,NaN,dedim,de,de,i said,say,267039,+dim,geçmiş,1.tekil,NaN,NaN,NaN,NaN
58,de,VERB,NaN,dedi,de,de,said,also,182523,+di,geçmiş,3.tekil,NaN,NaN,NaN,NaN
66,var,VERB,NaN,vardı,var,var,there was,there is,451341,+dı,geçmiş,NaN,NaN,NaN,NaN,to be
69,var,VERB,NaN,varmış,var,var,there was,there is,118304,+mış,geçmiş,NaN,NaN,NaN,NaN,to be
77,değil,VERB,NEG,değildi,değil,değil,it wasn't,not,235756,+di,geçmiş,NaN,NaN,NaN,olumsuz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,bulduk,VERB,NaN,bulduk,bul,bul,we found,find,77535,+,geçmiş,1.çoğul,NaN,NaN,NaN,NaN
1541,yapmış,VERB,NaN,yapmış,yap,yap,made,do it,73719,+,geçmiş,3.tekil,NaN,NaN,NaN,NaN
1600,istemedim,VERB,NEG,istemedim,iste,iste,i didn't want,want,68907,+,geçmiş,1.tekil,NaN,NaN,olumsuz,NaN
1610,bul,VERB,NaN,buldum,bul,bul,i found,find,179420,+dum,geçmiş,1.tekil,NaN,NaN,NaN,NaN


In [53]:
past_word_list = list(set(df_past_tense_result["word"]))
#past_word_list

In [54]:
for past_word in past_word_list:
    source = f"{word_video_path_folder}/{past_word}.mp4"
    destination = past_output_path
    shutil.copy2(source, destination)

###### Present

In [55]:
# ///***
add_present_out_folder = "Şimdiki"

present_output_path = f"{output_path}/{add_present_out_folder}"

Path(present_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [56]:
df_present_tense_result = df_deployment_word_remain[df_deployment_word_remain["tense"] == "şimdiki"]
df_present_tense_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
185,bile,VERB,NEG,bilemiyorum,bil,bil,i do not know,know,82742,+miyorum,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
201,biliyorum,VERB,NaN,biliyorum,bil,bil,i know,know,935083,+,şimdiki,1.tekil,NaN,NaN,NaN,NaN
249,bilmiyorum,VERB,NEG,bilmiyorum,bil,bil,i do not know,know,700046,+,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
281,biliyor,VERB,NaN,biliyorum,bil,bil,i know,know,935083,+um,şimdiki,1.tekil,NaN,NaN,NaN,NaN
285,biliyor,VERB,NaN,biliyoruz,bil,bil,we know,know,102953,+uz,şimdiki,1.çoğul,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,çalışıyorsun,VERB,NaN,çalışıyorsun,çalış,çalış,you work,work,77707,+,şimdiki,2.tekil,NaN,NaN,NaN,NaN
1498,anlamı,VERB,NEG,anlamıyorum,anla,anla,i don't understand,understand,91669,+yorum,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
1508,istemiyor,VERB,NEG,istemiyorum,iste,iste,i don't want,want,345472,+um,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
1575,istiyorsunuz,VERB,NaN,istiyorsunuz,iste,iste,you want,want,71208,+,şimdiki,2.çoğul,NaN,NaN,NaN,NaN


In [57]:
present_word_list = list(set(df_present_tense_result["word"]))
#present_word_list

In [58]:
for present_word in present_word_list:
    source = f"{word_video_path_folder}/{present_word}.mp4"
    destination = present_output_path
    shutil.copy2(source, destination)

###### Future

In [59]:
# ///***
add_future_out_folder = "Gelecek"

future_output_path = f"{output_path}/{add_future_out_folder}"

Path(future_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [60]:
df_future_tense_result = df_deployment_word_remain[df_deployment_word_remain["tense"] == "gelecek"]
df_future_tense_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
228,olacak,VERB,NaN,olacaksın,ol,ol,you will be,die,95544,+sın,gelecek,2.tekil,NaN,NaN,NaN,to be
324,ol,VERB,NaN,olacağım,ol,ol,i will be,be,150030,+acağım,gelecek,1.tekil,NaN,NaN,NaN,to be
333,ol,VERB,NaN,olacaksın,ol,ol,you will be,die,95544,+acaksın,gelecek,2.tekil,NaN,NaN,NaN,to be
383,ver,VERB,NaN,vereceğim,ver,ver,i will give,give,73483,+eceğim,gelecek,1.tekil,NaN,NaN,NaN,NaN
483,al,VERB,NaN,alacağım,al,al,i will take,get,82272,+acağım,gelecek,1.tekil,NaN,NaN,NaN,NaN
758,yap,VERB,NaN,yapacağım,yap,yap,i will do it,do it,135540,+acağım,gelecek,1.tekil,NaN,NaN,NaN,NaN
760,yap,VERB,NaN,yapacağız,yap,yap,we will do,do it,120078,+acağız,gelecek,1.çoğul,NaN,NaN,NaN,NaN
765,yap,VERB,NaN,yapacaksın,yap,yap,you will do,do it,96444,+acaksın,gelecek,2.tekil,NaN,NaN,NaN,imperative
974,olacağım,VERB,NaN,olacağım,ol,ol,i will be,be,150030,+,gelecek,1.tekil,NaN,NaN,NaN,to be
1046,yapacağım,VERB,NaN,yapacağım,yap,yap,i will do it,do it,135540,+,gelecek,1.tekil,NaN,NaN,NaN,NaN


In [61]:
future_word_list = list(set(df_future_tense_result["word"]))
#future_word_list

In [62]:
for future_word in future_word_list:
    source = f"{word_video_path_folder}/{future_word}.mp4"
    destination = future_output_path
    shutil.copy2(source, destination)

##### Negative

In [66]:
# ///***
add_negative_out_folder = "Olumsuz"

negative_output_path = f"{output_path}/{add_negative_out_folder}"

Path(negative_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [67]:
df_negative_mean_result = df_deployment_word_remain[df_deployment_word_remain["mean"] == "olumsuz"]
df_negative_mean_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
76,değil,VERB,NEG,değilim,değil,değil,i am not,not,448632,+im,NaN,1.tekil,NaN,NaN,olumsuz,NaN
77,değil,VERB,NEG,değildi,değil,değil,it wasn't,not,235756,+di,geçmiş,NaN,NaN,NaN,olumsuz,NaN
79,değil,VERB,NEG,değildir,değil,değil,it is not,not,126096,+dir,NaN,NaN,NaN,NaN,olumsuz,NaN
80,değil,VERB,NEG,değiliz,değil,değil,we are not,not,82899,+iz,NaN,1.çoğul,NaN,NaN,olumsuz,NaN
103,yok,VERB,NEG,yoktu,yok,yok,there was not,there is not,181801,+tu,geçmiş,NaN,NaN,NaN,olumsuz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,olma,VERB,NEG,olmasın,ol,ol,not to be,be,77684,+sın,NaN,NaN,NaN,NaN,olumsuz,to be
1493,olmasın,VERB,NEG,olmasın,ol,ol,not to be,be,77684,+,NaN,2.tekil,NaN,NaN,olumsuz,to be
1498,anlamı,VERB,NEG,anlamıyorum,anla,anla,i don't understand,understand,91669,+yorum,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
1508,istemiyor,VERB,NEG,istemiyorum,iste,iste,i don't want,want,345472,+um,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN


In [68]:
negative_word_list = list(set(df_negative_mean_result["word"]))
#negative_word_list

In [69]:
for negative_word in negative_word_list:
    source = f"{word_video_path_folder}/{negative_word}.mp4"
    destination = negative_output_path
    shutil.copy2(source, destination)

#### Result Video File Concat

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import shutil
from os.path import isfile, join
import time
import nltk
from nltk import word_tokenize

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

filename_length_num = 5

In [ ]:
def concat_video2(text_list, video_data_folder_name, video_data_path, out_folder_name, out_path, filename):  
    '''
    concat_video(text_list, video_data_folder_name, video_data_path, out_folder_name, out_path, filename)\n
    text_list is list values. video_data_folder_name, video_data_path, out_folder_name and out_path are path and folder in string(str)\n
    filename is string of result mp4 file name.
    ex.\n
    path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/200 Words"\n
    audio_image_video_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/200 Words"\n
    concat_video(word_select_list, "Image_Video_Audio", audio_image_video_path, "Concat_Video", path, "200_Word_Combined_Sample")
    '''
    no_image_file_list = []
    result_path = f"{out_path}/{out_folder_name}"
    Path(result_path).mkdir(parents=True, exist_ok=True)
    video_text_list = []
    video_list = []

    if isfile(f"{result_path}/{filename}.mp4"):
        pass
    else:
        for text in text_list:
            if isfile(f"{video_data_path}/{video_data_folder_name}/{text}.mp4"):
                video_text_list.append(text)            
            else:
                no_image_file_list.append(text)

        for video_num in range(len(video_text_list)):
            try:
                clip_var = VideoFileClip(f"{video_data_path}/{video_data_folder_name}/{video_text_list[video_num]}.mp4", target_resolution=(900, 1600))  # height, width
                video_list.append(clip_var)            
            except:
                print("Oops!", sys.exc_info()[0], "occurred.", f"{video_text_list[video_num]}.mp4")

        combined = concatenate_videoclips(video_list, method='compose')
        combined.write_videofile(f"{result_path}/{filename}.mp4", audio_bitrate='128k', audio_fps=44100, audio_codec="aac")
    
    return no_image_file_list

In [ ]:
# Previous output path and folder
#prev_result_path = f"/home/kurubal/Documents/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/200 Words"
#prev_main_result_folder = "Twogram_Threegram"
#prev_result_folder = "Threegram_Twogram_Selected"

prev_result_path = f"{result_path}"
prev_main_result_folder = f"{main_result_folder}"
prev_result_folder = f"{result_folder}"

In [ ]:
data_path = f"{prev_result_path}/{prev_main_result_folder}/{prev_result_folder}/Result"  # previous output path result for data for this
result_path = f"{prev_result_path}/{prev_main_result_folder}/{prev_result_folder}/Result"

In [ ]:
#df_deployment_file = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/\
#Deployment2/Result/Deploy2 Result Manuel/Turkish/Twogram_Threegram_Selected.xlsx", sheet_name="twogram_threegram_selected")
#df_deployment_file

In [ ]:
#word_group_list = df_deployment_file["twogram_threegram_selected"].dropna().to_list()
#word_group_list

In [ ]:
len(word_group_list)

In [ ]:
word_group_list2 = []
for word_group in word_group_list:
    word_list = word_tokenize(word_group)
    if len(word_list)>filename_length_num:
        filename_new = " ".join(word_list[0:filename_length_num])
    else:
        filename_new = word_group
    word_group_list2.append(filename_new)

In [ ]:
concat_video2(word_group_list2, "Concat_Video", data_path, "All_Concat_Video", result_path, f"{prev_result_folder}")